# RAG application built on gemini 

In [1]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("CerboTech Chatbot doc (3).pdf")
data = loader.load()  # entire PDF is loaded as a single Document
# data

In [2]:
len(data)

35

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# split data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)


print("Total number of documents: ",len(docs))

Total number of documents:  94


In [4]:
docs[3]

Document(metadata={'source': 'CerboTech Chatbot doc (3).pdf', 'page': 1}, page_content='Game\n \nMode:\n \nThoughtful\n \nKeywords:\n \nnumbers,\n \npuzzle,\n \nthinking,\n \nlogic,\n \nstrategy ,\n \ndynamic,\n \nchallenge,\n \ncognitive\n \nflexibility ,\n \nproblem-solving,\n \ndecision-making,\n \nprocessing,\n \nhigh\n \nscore\n \n \nGame\n \n2:\n \nAlphaQuest\n \nTagline:\n \nUnlock\n \nthe\n \nWord\n \nwith\n \nthe\n \nFewest\n \nMoves!\n \nDescription:\n \nUncover\n \nhidden\n \npatterns\n \nand\n \ncomplete\n \nthe\n \nquest!\n \nThis\n \ngame\n \ntests\n \nyour\n \nstrategic\n \nthinking\n \nand\n \npattern\n \nrecognition\n \nskills,\n \nwhile\n \nimproving\n \ncognitive\n \nflexibility .\n \nImproves:\n \nQuick\n \nThinking,\n \nProblem-Solving\n \nGame\n \nType:\n \nAlphabet\n \nSkill\n \nFocus:\n \nLogic\n \nGame\n \nMode:\n \nFun\n \nKeywords:\n \nwords,\n \nalphabet,\n \npuzzle,\n \npatterns,\n \nstrategy ,\n \nlogic,\n \nquick\n \nthinking,\n \nproblem-solving,\n \ncog

In [5]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from dotenv import load_dotenv
load_dotenv() 

#Get an API key: 
# Head to https://ai.google.dev/gemini-api/docs/api-key to generate a Google AI API key. Paste in .env file

# Embedding models: https://python.langchain.com/v0.1/docs/integrations/text_embedding/

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("hello, world!")
vector[:5]
#vector

C:\Users\sneha\anaconda3\envs\langchain_gemini_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[0.05168594419956207,
 -0.030764883384108543,
 -0.03062233328819275,
 -0.02802734263241291,
 0.01813093200325966]

In [6]:
vectorstore = Chroma.from_documents(documents=docs, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

In [7]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})

retrieved_docs = retriever.invoke("What is NeurOm?")


In [8]:
len(retrieved_docs)

10

In [9]:
print(retrieved_docs[5].page_content)

here
 
to
 
help
 
you
 
discover
 
the
 
games
 
and
 
activities
 
that
 
best
 
suit
 
your
 
current
 
needs,
 
whether
 
you're
 
looking
 
to
 
de-stress,
 
sharpen
 
your
 
mind,
 
find
 
a
 
particular
 
breathing
 
technique,
 
or
 
just
 
explore
 
what
 
we
 
offer.
 
Q10:
 
Does
 
NeurOm
 
track
 
my
 
progress?
 
A10:
 
Yes,
 
NeurOm
 
helps
 
you
 
keep
 
track
 
of
 
your
 
journey!
 
You
 
can
 
visit
 
the
 
"My
 
Performance"
 
section
 
to
 
see
 
things
 
like
 
your
 
total
 
Points
 
Rewarded
 
–
 
we
 
award
 
specific
 
points
 
for
 
each
 
game
 
and
 
activity
 
you
 
complete
 
–
 
your
 
Rank
 
Achieved
,
 
how
 
many
 
activities
 
you've
 
performed,
 
and
 
games
 
you've
 
played.
 
It's
 
a
 
great
 
way
 
to
 
see
 
your
 
engagement
 
and
 
achievements!
 
Q11:
 
Are
 
there
 
different
 
difficulty
 
levels
 
for
 
the
 
games?
 
A11:
 
Many
 
of
 
our
 
games,
 
like
 
NumberStorm
 
or
 
StackUp,
 
naturally
 
increase
 
in
 
difficulty
 
as
 
you


In [10]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")

In [11]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ] 
)

In [12]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [13]:
response = rag_chain.invoke({"input": " what is the Core Purpose"}) 
print(response["answer"])

NeurOm's core purpose is to help users improve their mental well-being and cognitive skills through mindful games and guided activities like meditation and focus exercises.  The app uses a combination of these methods to achieve this goal.
